<a href="https://colab.research.google.com/github/freedrobili/302_OOP_Kudashkina_JuE/blob/master/Lab_1_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab

/content/drive/MyDrive/Colab


# **Задание 1 - Проработка примеров**

---



# **Пример 3.1**

---

In [ ]:
%%file primer_1.cpp

#include <iostream>
#include<omp.h>
#include<stdio.h>

int main () {
    int size, rank;

    /* Создание множества параллельных процессов и в каждом из них задаются
    * свои приватные переменные size и rank */

    #pragma omp parallel private(size, rank)
    {
        /* Каждый процесс находит свой порядковый номер и выводит его на экран */

        rank = omp_get_thread_num();
        printf("Hello World from thread = %d\n", rank);

        /* Главный процесс - master выводит на экран количество процессов */

        if (rank == 0) {
            size = omp_get_num_threads();
            printf("Number of threads = %d\n", size);
        }

    } /* Завершение параллельной части */

    return 0;
}

Overwriting primer_1.cpp


In [ ]:
!g++ primer_1.cpp -fopenmp -o primer_1

In [ ]:
!./primer_1

Hello World from thread = 0
Number of threads = 2
Hello World from thread = 1


# **Пример 3.2**

---



In [ ]:
%%file primer_2.cpp

#include <iostream>
#include<omp.h>
#include<stdio.h>

int main () {
    int i, n;
    float a[100], b[100], sum;

    /* Инициализация элементов векторов */

    n = 100;
    for (i=0; i < n; i++)
        a[i] = b[i] = i * 1.0;
    sum = 0.0;

    /* Создание множества параллельных процессов и распараллеливание
    * цикла по виткам. При выходе из цикла все значения переменной sum
    * суммируются по всем процессам. */

    #pragma omp parallel for reduction(+:sum)
    for (i=0; i < n; i++)
        sum = sum + (a[i] * b[i]);

    /* Главный процесс выводит на экран значение sum */

    printf(" Sum = %f\n",sum);

    return 0;
}

Overwriting primer_2.cpp


In [ ]:
!g++ primer_2.cpp -fopenmp -o primer_2

In [ ]:
!./primer_2

 Sum = 328350.000000


# **Пример 3.3**

---



In [ ]:
%%file primer_3.cpp

#include <iostream>
#include<omp.h>
#include<stdio.h>

#define VECLEN 100
float a[VECLEN], b[VECLEN], sum;

/* Подпрограмма, в которой суммируются элементы векторов */

float dotprod () {
    int i,rank;
    rank = omp_get_thread_num();
    #pragma omp for reduction(+:sum)
    for (i=0; i < VECLEN; i++) {
        sum = sum + (a[i]*b[i]);
        printf(" rank = %d i=%d\n",rank,i);
    }
    return(sum);
}

int main () {
  int i;

  /* Инициализация элементов векторов */

  for (i=0; i < VECLEN; i++)
      a[i] = b[i] = 1.0 * i;
  sum = 0.0;

  /* Создание множества параллельных процессов */

  #pragma omp parallel
  sum = dotprod();
  printf("Sum = %f\n",sum);

  return 0;
}


Writing primer_3.cpp


In [ ]:
!g++ primer_3.cpp -fopenmp -o primer_3

In [ ]:
!./primer_3

 rank = 0 i=0
 rank = 0 i=1
 rank = 0 i=2
 rank = 0 i=3
 rank = 0 i=4
 rank = 0 i=5
 rank = 1 i=50
 rank = 1 i=51
 rank = 1 i=52
 rank = 1 i=53
 rank = 1 i=54
 rank = 1 i=55
 rank = 1 i=56
 rank = 1 i=57
 rank = 1 i=58
 rank = 1 i=59
 rank = 1 i=60
 rank = 1 i=61
 rank = 1 i=62
 rank = 1 i=63
 rank = 1 i=64
 rank = 1 i=65
 rank = 1 i=66
 rank = 1 i=67
 rank = 1 i=68
 rank = 1 i=69
 rank = 1 i=70
 rank = 1 i=71
 rank = 1 i=72
 rank = 1 i=73
 rank = 1 i=74
 rank = 1 i=75
 rank = 1 i=76
 rank = 1 i=77
 rank = 1 i=78
 rank = 1 i=79
 rank = 1 i=80
 rank = 1 i=81
 rank = 1 i=82
 rank = 1 i=83
 rank = 1 i=84
 rank = 1 i=85
 rank = 1 i=86
 rank = 1 i=87
 rank = 1 i=88
 rank = 1 i=89
 rank = 1 i=90
 rank = 1 i=91
 rank = 1 i=92
 rank = 1 i=93
 rank = 1 i=94
 rank = 1 i=95
 rank = 1 i=96
 rank = 1 i=97
 rank = 1 i=98
 rank = 1 i=99
 rank = 0 i=6
 rank = 0 i=7
 rank = 0 i=8
 rank = 0 i=9
 rank = 0 i=10
 rank = 0 i=11
 rank = 0 i=12
 rank = 0 i=13
 rank = 0 i=14
 rank = 0 i=15
 rank = 0 i=16
 rank

# **Пример 3.4**

---



In [ ]:
%%file primer_4.cpp

#include <iostream>
#include<omp.h>
#include<stdio.h>

#define N 50
int main () {
    int i, size, rank;
    float a[N], b[N], c[N];

    /* Инициализация элементов векторов */

    for (i=0; i < N; i++)
        a[i] = b[i] = i * 1.0;

    /* Создание множества параллельных процессов */

    #pragma omp parallel shared(a,b,c) private(i,rank,size)
    {
        /* Каждый процесс находит свой порядковый номер и выводит его на экран */

        rank = omp_get_thread_num();
        printf("Thread %d starting...\n",rank);

        /* Директива задания секций */

        #pragma omp sections nowait
        {
            /* Секция 0*/

            #pragma omp section
            for (i=0; i < N/2; i++) {
                c[i] = a[i] + b[i];
                printf("rank = %d i= %d c[i]= %f\n", rank,i,c[i]);
            }

            /* Секция 1*/

            #pragma omp section
            for (i=N/2; i < N; i++) {
                c[i] = a[i] + b[i];
                printf("rank = %d i= %d c[i]= %f\n", rank,i,c[i]);
            }

        } /* Завершение блока секций */

        if (rank == 0) {
          size = omp_get_num_threads();
          printf("Number of threads = %d\n", size);
        }

    } /* Завершение параллельной части */

    return 0;
}


Overwriting primer_4.cpp


In [ ]:
!g++ primer_4.cpp -fopenmp -o primer_4

In [ ]:
!./primer_4

Thread 1 starting...
rank = 1 i= 0 c[i]= 0.000000
rank = 1 i= 1 c[i]= 2.000000
rank = 1 i= 2 c[i]= 4.000000
rank = 1 i= 3 c[i]= 6.000000
rank = 1 i= 4 c[i]= 8.000000
rank = 1 i= 5 c[i]= 10.000000
rank = 1 i= 6 c[i]= 12.000000
rank = 1 i= 7 c[i]= 14.000000
rank = 1 i= 8 c[i]= 16.000000
rank = 1 i= 9 c[i]= 18.000000
rank = 1 i= 10 c[i]= 20.000000
rank = 1 i= 11 c[i]= 22.000000
rank = 1 i= 12 c[i]= 24.000000
rank = 1 i= 13 c[i]= 26.000000
rank = 1 i= 14 c[i]= 28.000000
rank = 1 i= 15 c[i]= 30.000000
rank = 1 i= 16 c[i]= 32.000000
rank = 1 i= 17 c[i]= 34.000000
rank = 1 i= 18 c[i]= 36.000000
rank = 1 i= 19 c[i]= 38.000000
rank = 1 i= 20 c[i]= 40.000000
rank = 1 i= 21 c[i]= 42.000000
rank = 1 i= 22 c[i]= 44.000000
rank = 1 i= 23 c[i]= 46.000000
rank = 1 i= 24 c[i]= 48.000000
rank = 1 i= 25 c[i]= 50.000000
rank = 1 i= 26 c[i]= 52.000000
rank = 1 i= 27 c[i]= 54.000000
rank = 1 i= 28 c[i]= 56.000000
rank = 1 i= 29 c[i]= 58.000000
rank = 1 i= 30 c[i]= 60.000000
rank = 1 i= 31 c[i]= 62.000000
ra

# **Пример 3.5**

---



In [ ]:
%%file primer_5.cpp

#include <iostream>
#include<omp.h>
#include<stdio.h>

#define M 10
int main () {
    float A[M][M], b[M], c[M];
    int i, j, rank;

    /* Инициализация данных */

    for (i=0; i < M; i++) {
        for (j=0; j < M; j++)
            A[i][j] = (j+1) * 1.0;
        b[i] = 1.0 * (i+1);
        c[i] = 0.0;
    }

    printf("Вывод значений матрицы A и вектора b на экран:\n");

    for (i=0; i < M; i++) {
        printf(" A[%d]=",i);
        for (j=0; j < M; j++)
            printf("%.1f ",A[i][j]);
        printf(" b[%d]=%.1f\n",i,b[i]);
    }

    /* Создание множества параллельных процессов и в каждом из них задаются
    * свои приватные переменные rank и i*/

    #pragma omp parallel shared(A,b,c) private(rank,i)
    {
        rank = omp_get_thread_num();

        /* Директива распараллеливания цикла по виткам */

        #pragma omp for private(j)
        for (i=0; i < M; i++) {
            for (j=0; j < M; j++)
                c[i] += (A[i][j] * b[j]);

            /* Каждый процесс выводит свой порядковый номер, значение витка цикла и
            * значение результирующего вектора на каждом витке цикла и внутри
            * критической секции */

            #pragma omp critical
            {
                printf(" rank=%d ", rank);
                printf("i=%d ", i);
                printf("c[%d]=%.2f\n", i, c[i]);
            }

        } /* Конец параллельного цикла */

    } /* Завершение параллельной конструкции */

    return 0;
}

Overwriting primer_5.cpp


In [ ]:
!g++ primer_5.cpp -fopenmp -o primer_5

In [ ]:
!./primer_5

Вывод значений матрицы A и вектора b на экран:
 A[0]=1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0  b[0]=1.0
 A[1]=1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0  b[1]=2.0
 A[2]=1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0  b[2]=3.0
 A[3]=1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0  b[3]=4.0
 A[4]=1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0  b[4]=5.0
 A[5]=1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0  b[5]=6.0
 A[6]=1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0  b[6]=7.0
 A[7]=1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0  b[7]=8.0
 A[8]=1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0  b[8]=9.0
 A[9]=1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0  b[9]=10.0
 rank=0 i=0 c[0]=385.00
 rank=1 i=5 c[5]=385.00
 rank=0 i=1 c[1]=385.00
 rank=1 i=6 c[6]=385.00
 rank=0 i=2 c[2]=385.00
 rank=1 i=7 c[7]=385.00
 rank=0 i=3 c[3]=385.00
 rank=1 i=8 c[8]=385.00
 rank=0 i=4 c[4]=385.00
 rank=1 i=9 c[9]=385.00


# **Задание 2 - Параллельное умножение матрицы на вектор**

---



In [ ]:
%%file lab_1_task2.cpp

#include <iostream>
#include<omp.h>
#include<stdio.h>

const int N = 30;
double **a, *b, *c;

int main() {
    int i, j, num_threads, p, n;

    a = new double*[N];
    b = new double[N];
    c = new double[N];

    for (i = 0; i < N; i++) {
        b[i] = 1;
        a[i] = new double[N];
        for (j = 0; j < N; j++) {
            a[i][j] = 1;
        }
    }

    std::cout << "Матрица А:" << std::endl;
    for (i = 0; i < N; i++) {
        for (j = 0; j < N; j++)
            std::cout << a[i][j] << "  ";
        std::cout << "\n";
    }

    std::cout << "\nВектор b:" << std::endl;
    for (i = 0; i < N; i++)
        std::cout << b[i] << "  ";

    std::cout << "\n";

    #pragma omp parallel shared(a, b, c) private(i, j, num_threads, p, n)
    {
        num_threads = omp_get_num_threads();
        p = omp_get_thread_num();
        n = N/num_threads;

        for (i = n*p; i < n*(p+1); i++) {
            c[i] = 0.0;
            for (j = 0; j < N; j++) {
                c[i] += a[i][j]*b[j];
            }
        }
    }

    std::cout << "\nРезультирующий вектор с:" << std::endl;
    for(i = 0; i < N; i++)
        std::cout << c[i] << "  ";

    return 0;
}

Overwriting lab_1_task2.cpp


In [ ]:
!g++ lab_1_task2.cpp -fopenmp -o lab_1_task2

In [ ]:
!./lab_1_task2

Матрица А:
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1

# **Задание 3 - Параллельное умножение матрицы на матрицу**

---



In [ ]:
%%file lab_1_task3.cpp

#include <iostream>
#include<omp.h>
#include<stdio.h>

const int N = 30;
double **a, **b, **c;

int main() {
    int i, j, k, num_threads, p, n;

    a = new double*[N];
    b = new double*[N];
    c = new double*[N];

    for (i = 0; i < N; i++) {
        a[i] = new double[N];
        b[i] = new double[N];
        c[i] = new double[N];
        for (j = 0; j < N; j++) {
            a[i][j] = 1;
            b[i][j] = 2;
        }
    }

    std::cout << "Матрица А:" << std::endl;
    for (i = 0; i < N; i++) {
        for (j = 0; j < N; j++)
            std::cout << a[i][j] << "  ";
        std::cout << "\n";
    }

    std::cout << "\nМатрица B:" << std::endl;
    for (i = 0; i < N; i++) {
        for (j = 0; j < N; j++)
            std::cout << b[i][j] << "  ";
        std::cout << "\n";
    }

    std::cout << "\n";

    #pragma omp parallel shared(a, b, c) private(i, j, k, num_threads, p, n)
    {
        num_threads = omp_get_num_threads();
        p = omp_get_thread_num();
        n = N/num_threads;

        for (i = p*n; i < n*(p+1); i++) {
            for (j = 0; j < N; j++) {
                c[i][j] = 0.0;
                for(k = 0; k < N; k++){
                    c[i][j] += a[i][k]*b[k][j];
                }
            }
        }
    }

    std::cout << "\nРезультирующая матрица С:" << std::endl;
    for (i = 0; i < N; i++) {
        for (j = 0; j < N; j++)
            std::cout << c[i][j] << "  ";
        std::cout << "\n";
    }

    return 0;
}

Overwriting lab_1_task3.cpp


In [ ]:
!g++ lab_1_task3.cpp -fopenmp -o lab_1_task3

In [ ]:
!./lab_1_task3

Матрица А:
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1